In [5]:
%%capture
%pip install -U datasets 
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [6]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
# from google.colab import userdata
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    EarlyStoppingCallback,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format, DataCollatorForCompletionOnlyLM
# wb_token = userdata.get('wandb')


2024-07-07 17:27:20.874657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 17:27:20.874782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 17:27:20.986273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)

run = wandb.init(
    project='Model 6', 
    job_type="training", 
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maithaoly2001 (rag1). Use `wandb login --relogin` to force relogin


In [8]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [9]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "model-6"

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj'],
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)
# model, tokenizer = setup_chat_format(model, tokenizer)
# model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [10]:
from peft import get_peft_model
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 8,051,249,152 || trainable%: 0.2605


In [11]:
import pandas as pd
from datasets import list_datasets, load_dataset,Dataset

data = pd.read_csv("/kaggle/input/dataft6b/message_data2.csv",header = None)
#data = data[1:]
data.columns = ['a']
# data = [eval(l) for l in list(data.iloc[:,0].values)]
dataset = Dataset.from_pandas(data)

# dataset  = load_dataset(data)

In [12]:
data

,a
0,"[{""role"": ""system"", ""content"": ""Task Descripti..."
1,"[{""role"": ""system"", ""content"": ""Task Descripti..."
2,"[{""role"": ""system"", ""content"": ""Task Descripti..."
3,"[{""role"": ""system"", ""content"": ""Task Descripti..."
4,"[{""role"": ""system"", ""content"": ""Task Descripti..."
...,...
900,"[{""role"": ""system"", ""content"": ""Task Descripti..."
901,"[{""role"": ""system"", ""content"": ""Task Descripti..."
902,"[{""role"": ""system"", ""content"": ""Task Descripti..."
903,"[{""role"": ""system"", ""content"": ""Task Descripti..."


In [13]:

dataset = dataset.shuffle(seed=42)#.select(range(1000))

def format_chat_template(row):
    row["a"] = tokenizer.apply_chat_template(eval(row["a"]), tokenize=False)
    # row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)
dataset = dataset.train_test_split(test_size=0.01)


/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/905 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['a'],
        num_rows: 895
    })
    test: Dataset({
        features: ['a'],
        num_rows: 10
    })
})

In [15]:
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
response_template = "\n<|im_start|>assistant"


# collator = DataCollatorForCompletionOnlyLM(response_template="### Response:", tokenizer=tokenizer, mlm=False)
response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[1:]
        
collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template_ids,
    tokenizer=tokenizer,
)
tokenizer.pad_token = tokenizer.eos_token

In [16]:
import time
from transformers import TrainerCallback, TrainerState, TrainerControl

class TimerCallback(TrainerCallback):
    def __init__(self, max_training_hours, max_training_minutes=0):
        self.max_training_time = (max_training_hours * 3600) + (max_training_minutes * 60)  # Convert hours and minutes to seconds
        self.start_time = None
    
    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()
    
    def on_step_end(self, args, state, control, **kwargs):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.max_training_time:
            control.should_training_stop = True
            print(f"Training stopped after {elapsed_time / 3600:.2f} hours.")

In [17]:
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl

class SaveLastCheckpointCallback(TrainerCallback):
    def on_train_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if state.global_step % args.save_steps != 0:
            state.save_model(args.output_dir)
            control.should_save = True
            print(f"Saving last checkpoint at step {state.global_step}")

In [18]:
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(

    output_dir=new_model,
    # max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    report_to="wandb",
    num_train_epochs=2,
    optim="paged_adamw_32bit",
    do_eval = True,
    logging_steps=20 ,
    evaluation_strategy="steps",
    logging_strategy="steps",
    learning_rate=2e-4,
    eval_steps=50,
    #save_steps=50,  # Lưu checkpoint mỗi 50 bước
    save_total_limit=1, # Giới hạn số lượng checkpoint lưu trữ
    load_best_model_at_end=True,
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Số lần không cải thiện trước khi dừng
    early_stopping_threshold=0.01  # Ngưỡng cải thiện (có thể bỏ qua)
)
max_training_hours = 11
max_training_minutes = 0 # Giới hạn số phút training
timer_callback = TimerCallback(max_training_hours, max_training_minutes)


trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    data_collator=collator,
    args=sft_config,
    dataset_text_field="a",
    train_dataset=dataset["train"], 
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    callbacks=[early_stopping_callback, timer_callback, SaveLastCheckpointCallback()]  # Thêm early stopping callback
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn

Map:   0%|          | 0/895 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.401500,0.451554
100,0.370000,0.420631
150,0.334200,0.411958
200,0.354300,0.396362
250,0.271500,0.397762
300,0.263600,0.393796
350,0.264500,0.388195
400,0.233200,0.385586


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


AttributeError: 'TrainerState' object has no attribute 'save_model'

In [19]:
!zip -r 'checkpoint-448'.zip '/kaggle/working/model-6/checkpoint-448'

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: kaggle/working/model-6/checkpoint-448/ (stored 0%)
  adding: kaggle/working/model-6/checkpoint-448/tokenizer.json (deflated 74%)
  adding: kaggle/working/model-6/checkpoint-448/adapter_model.safetensors (deflated 52%)
  adding: kaggle/working/model-6/checkpoint-448/trainer_state.json (deflated 78%)
  adding: kaggle/working/model-6/checkpoint-448/README.md (deflated 66%)
  adding: kaggle/working/model-6/checkpoint-448/adapter_config.json (deflated 54%)
  adding: kaggle/working/model-6/checkpoint-448/optimizer.pt (deflated 9%)
  adding: kaggle/working/model-6/checkpoint-448/scheduler.pt (deflated 57%)
  adding: kaggle/working/model-6/checkpoint-448/training_args.bin (deflated 51%)
  adding: kaggle/working/model-6/checkpoint-448/rng_state.pth (deflated 25%)
  adding: kaggle/working/model-6/checkpoint-448/tokenizer_config.json (deflated 96%)
  adding: kaggle/working/model-6/checkpoint-448/special_tokens_map.json (deflated 66%)


In [20]:
from IPython.display import FileLink
FileLink(r'checkpoint-448.zip')

/kaggle/working/checkpoint-448.zip